In [ ]:
from clease.structgen import NewStructures
from clease.settings import CEBulk, Concentration

conc = Concentration(basis_elements=[["Ti", "Al", "Nb", "Mo", "Zr"]])

settings = CEBulk(
    concentration=conc,
    crystalstructure="bcc",
    a=3.31,
    db_name="clease.db",
    max_cluster_dia=(
        8.63179,
        5.796069999999999,
        4.751329999999999,
        4.751329999999999,
        4.751329999999999,
    ),
)

In [ ]:
import os
from ase.db import connect
from ase.db.row import AtomsRow
from ase.calculators.singlepoint import SinglePointCalculator

new_structures = NewStructures(settings)

chemical_symbols = ["Ti", "Al", "Nb", "Mo", "Zr"]
# chemical_symbols = ["Ti", "Al", "Nb", "Mo"]
# chemical_symbols = ["Ti", "Al", "Mo", "Zr"]
system_name = "".join(chemical_symbols)

home_name = os.path.expanduser("~")
db_filepath = f"scripts/HEI-CE-MC/0-training-data/{system_name}_1_8.db"
if "mseklt" in home_name:
    db_filepath = os.path.join(home_name, "yangsl", db_filepath)
else:
    db_filepath = os.path.join(home_name, db_filepath)

imported_db = connect(db_filepath)
for row in imported_db.select("nowarning=Yes,structure_id<=ICET-Training-No-02474"):
    row: AtomsRow
    energy_total = row.energy_total

    atoms = row.toatoms()
    final_atoms = atoms.copy()
    results = {"energy": energy_total}
    final_atoms.calc = SinglePointCalculator(atoms, **results)

    ini_id = new_structures.insert_structure(
        init_struct=atoms, final_struct=final_atoms
    )

In [ ]:
from clease import Evaluate

eva = Evaluate(settings=settings, scoring_scheme="loocv", nsplits=10)

eva.set_fitting_scheme(fitting_scheme="lasso")
alpha = eva.plot_CV(alpha_min=1e-6, alpha_max=1.0, num_alpha=50)
print("the chosen value of alpha is")
print(alpha)

# set the alpha value with the one found above, and fit data using lasso regression
eva.set_fitting_scheme(fitting_scheme="lasso", alpha=alpha)
eva.plot_fit(interactive=False)

# plot ECI values
eva.plot_ECI()